In [1]:
%gui qt
import os.path as op
import mne

sample_path = mne.datasets.sample.data_path()
data_path = op.join(sample_path, 'MEG', 'sample')
subjects_dir = op.join(sample_path, 'subjects')

In [2]:
inv = mne.minimum_norm.read_inverse_operator(op.join(data_path, 'sample_audvis-meg-oct-6-meg-fixed-inv.fif'))

In [3]:
raw = mne.io.read_raw_fif(op.join(data_path, 'sample_audvis_filt-0-40_raw.fif'))
events = mne.read_events(op.join(data_path, 'sample_audvis_filt-0-40_raw-eve.fif'))
event_id = {'audio/left': 1,
            'audio/right': 2,
            'visual/left': 3,
            'visual/right': 4}
epochs = mne.Epochs(raw, events, event_id)

In [4]:
src = mne.read_source_spaces(op.join(subjects_dir, 'sample', 'bem', 'sample-all-src.fif'))

In [5]:
epochs_stc = mne.minimum_norm.apply_inverse_epochs(epochs, inv, lambda2=0.1111)

In [6]:
import rsa
import numpy as np

In [7]:
model_dsm = np.array([
    [0, 0.5, 1, 1],
    [0.5, 0, 1, 1],
    [1, 1, 0, 0.5],
    [1, 1, 0.5, 0],
])
rsa_vals = rsa.rsa_source_level(epochs_stc, model_dsm, inv['src'], y=epochs.events[:, 2], n_folds=5, n_jobs=4, verbose=True, stc_dsm_metric='sqeuclidean')

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  2.1min remaining:  2.1min
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  2.2min finished


In [8]:
rsa_vals.plot('sample', hemi='both', subjects_dir=subjects_dir)

<Brain subject_id="sample", hemi="both", surf="inflated">
